In [48]:
# imports
import torch
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F
from utils import *
import pandas as pd
import pickle
import numpy as np
from torch import nn
from torch.utils.data import DataLoader, Dataset

In [17]:
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/LaBSE")
model = AutoModel.from_pretrained("sentence-transformers/LaBSE").eval()

Some weights of the model checkpoint at sentence-transformers/LaBSE were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
srt = ["source", "reference", "translation"]
language_pairs = [
    "cs-en",
    "de-en",
    "en-fi",
    "en-zh",
    "ru-en",
    "zh-en",
]

In [26]:
scores = {pair: pd.read_csv(f"corpus/{pair}/scores.csv") for pair in language_pairs}

In [8]:
tokens = {pair: {t: tokenizer(scores[pair][t].tolist(), return_tensors="pt", padding=True, max_length=512) for t in srt} for pair in language_pairs}

In [34]:
pair = "cs-en"
embedding_ref = torch.from_numpy(np.load(f"corpus/{pair}/laser.reference_embeds.npy"))
embedding_src = torch.from_numpy(np.load(f"corpus/{pair}/laser.source_embeds.npy"))
embedding_trn = torch.from_numpy(np.load(f"corpus/{pair}/laser.translation_embeds.npy"))
score = torch.tensor(scores[pair]["z-score"])

In [67]:
torch.split(embedding_ref, 2, 1)[0].shape

torch.Size([11585, 2])

In [52]:
torch.cat((embedding_ref, embedding_trn),1).shape

torch.Size([11585, 2048])

In [22]:
from fnet import FNet
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [19]:
sentences = ["Hello World", "Hallo Welt"]
sentence_output = get_embeddings(sentences, tokenizer, model, False)

In [25]:
torch.tensor(embedding, device=device).shape

torch.Size([11585, 1024])

In [16]:
model = FNet()

In [ ]:
model()

tensor([[ 0.0162, -0.0006, -0.0060,  ...,  0.0321,  0.0054,  0.0290],
        [ 0.0083,  0.0660,  0.0101,  ...,  0.0179,  0.0370,  0.0303],
        [ 0.0113, -0.0009,  0.0086,  ...,  0.0049,  0.0101,  0.0339],
        ...,
        [ 0.0451, -0.0005,  0.0089,  ...,  0.0177,  0.0040,  0.0245],
        [ 0.0164,  0.0023,  0.0007,  ...,  0.0274,  0.0228,  0.0193],
        [ 0.0200,  0.0163,  0.0004,  ...,  0.0027,  0.0321,  0.0322]])

In [9]:
for pair in language_pairs:
    for t in srt:
        with open(f"tokens/LaBSE_tokens_{pair}_{t}.pkl", "wb") as f:
            pickle.dump(tokens[pair][t], f)

In [10]:
with open(f"tokens/LaBSE_tokens_cs-en_reference.pkl", "rb") as f:
    token = pickle.load(f)

In [2]:
token

NameError: name 'token' is not defined

In [ ]:
embedding_ = get_embeddings(scores["cs-en"][srt[0]].tolist(), tokenizer, model)

In [ ]:
for pair in language_pairs:
    for t in srt:
        embeddings = get_embeddings(scores[pair][t].tolist(), tokenizer, model)
        
        with open(f"tokens/LaBSE_embeddings_{pair}_{t}.pkl", "wb") as f:
            pickle.dump(embeddings, f)
        del embeddings

In [5]:
embeddings = {pair: {t: get_embeddings(scores[pair][t].tolist(), tokenizer, model) for t in srt} for pair in language_pairs}

KeyboardInterrupt: 

In [48]:
with open(f"tokens/LaBSE_embeddings.pkl", "rb") as f:
    embeddings = pickle.load(f)